# Core Imports

In [ ]:
# Custom Imports
from polysaccharide import general
from polysaccharide.general import optional_in_place
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdcompare, rdconvert, rdkdraw, rdcompare, rdprops, rdbond, rdlabels

from polysaccharide.polymer import monomer as monoutils
from polysaccharide.polymer.monomer import MonomerInfo
from polysaccharide.polymer.management import PolymerManager

from polysaccharide.polymer import building
import mbuild as mb

# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

# File and chemistry type definitions

In [135]:
pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

lammps_path = Path('LAMMPS')
lammps_path.mkdir(exist_ok=True)

omm_path = Path('OpenMM')
omm_path.mkdir(exist_ok=True)

In [ ]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Building PDB files from monomer fragments

In [ ]:
from polysaccharide.polymer.exceptions import SubstructMatchFailedError

DOP = 10

failed = general.RecursiveDict()
for chemistry in chemistries:
    mono_src_dir = MONO_INFO_DIR / chemistry
    pdb_out_dir = pdb_path / chemistry
    pdb_out_dir.mkdir(exist_ok=True)

    for mono_path in mono_src_dir.iterdir():
        mono_info = MonomerInfo.from_file(mono_path)
        try:
            polymer = building.build_linear_polymer(mono_info.SMARTS, DOP=DOP, sequence='AB')  
            savepath = pdb_out_dir / f'{mono_path.stem}.pdb'
            polymer.save(general.asstrpath(savepath))
        except Exception as e:
            failed[chemistry][e.__class__.__name__][mono_path.stem] = mono_info

In [ ]:
failed

## Testing sanitization of monomers prior to mbuild conversion

In [ ]:
rdkdraw.set_rdkdraw_size(500, 3/2)
cvtr = rdconvert.SMILESConverter()

chemistry = 'urethane'
idx = 10

mono_info = MonomerInfo.from_file(MONO_INFO_DIR / chemistry / f'{chemistry}_{idx}.json')

mbmols = {}
def show(mono_info : MonomerInfo) -> None:
    for resname, smarts in mono_info.SMARTS.items():
        print(resname, smarts)
        mol = Chem.MolFromSmarts(smarts)
        mol = cvtr.convert(mol)
        Chem.SanitizeMol(mol)
        rdlabels.clear_atom_map_nums(mol, in_place=True)
        display(mol)

        # mbmol, ports = building.mbmol_from_mono_smarts(new_smarts)

        prot_mol = rdbond.hydrogenate_rdmol_ports(mol, in_place=False)
        mbmols[resname] = mb.conversion.from_rdkit(prot_mol)

show(mono_info)

# Collating urethanes into collections and generating Interchange files

## Populating collections from newly-generated PDBs and monomers

In [ ]:
for chemistry in chemistries:
    chem_path = coll_path / chemistry
    chem_path.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    mgr.populate_collection(pdb_path / chemistry, MONO_INFO_DIR / chemistry)

## Generating LAMMPS files via Interchange for both collections

In [ ]:
from polysaccharide import OPENFF_DIR
from polysaccharide.charging.application import MolCharger

# specify forcefield
ff_name = 'openff-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# specify charging method
chg_method = 'Espaloma_AM1BCC'

In [ ]:
from tqdm.notebook import tqdm

success_ics = defaultdict(defaultdict)
failed_ics  = defaultdict(lambda : defaultdict(list))

for chemistry in chemistries:
    chem_path = coll_path / chemistry
    lmp_dir = lammps_path / chemistry
    lmp_dir.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    for mol_name, polymer in ( progress := tqdm(mgr.polymers.items()) ):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        try:
            chgr = MolCharger.subclass_registry[chg_method]()
            polymer.assert_charges_for(chgr, strict=True, return_cmol=False)

            sdf_path = polymer.structure_files_chgd[chg_method]
            cmol = polymer.charged_offmol_from_sdf(chg_method)
            offtop = Topology.from_molecules(cmol) # load topology from SDF file

            ic = forcefield.create_interchange(offtop, charge_from_molecules=[cmol])
            success_ics[chemistry][mol_name] = ic
            # ic.to_lammps(lmp_dir / f'{mol_name}.lammps')

        except AttributeError as a:
            print(a)

        except Exception as e:
            print(e)
            failed_ics[chemistry][e.__class__.__name__].append(mol_name)

In [ ]:
success_ics, failed_ics

# Running OpenMM simulations

In [136]:
import re
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

from polysaccharide.simulation import preparation
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory

STRIP_BEFORE_PARENS = re.compile(r'(.*?)(?=\s*\(.*\))')
PRECISION = 3 # number of decimals to round reported energies to


# selecting simulation parameters and ensemble
sp_path = Path('debug_sim_NVT.json')
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

data_by_mol = []
# looping over all urethanes
for chemistry, ic_dict in success_ics.items():
    chem_dir = omm_path / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, interchange in ic_dict.items():
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        sim_file_dir = mol_dir / f'{mol_name}_sim'
        sim_file_dir.mkdir(exist_ok=True, parents=True)

        # creating simulation and associated files
        sim = ens_fac.create_simulation(interchange, sim_params)
        sim_paths = preparation.prepare_simulation_paths(output_folder=sim_file_dir, output_name=mol_name, sim_params=sim_params)
        reporters = preparation.prepare_simulation_reporters(sim_paths, sim_params)
        preparation.config_simulation(sim, reporters, checkpoint_path=sim_paths.checkpoint)

        # energy min and single-step integration
        sim.minimizeEnergy()
        sim.step(1)

        # extracting energies

        state_data = pd.read_csv(sim_paths.state_data)
        energies = {}
        for key in ('Potential Energy (kJ/mole)', 'Kinetic Energy (kJ/mole)'):
            tag = re.search(STRIP_BEFORE_PARENS, key).group(0)
            E_kj_val = state_data[key][0]
            E_kj = E_kj_val * kilojoule_per_mole
            E_kcal = E_kj.in_units_of(kilocalorie_per_mole)

            for energy in (E_kj, E_kcal):
                energies[f'{tag} ({energy.unit.get_symbol()})'] = energy._value
        
        data_dict.update(**energies)
        data_by_mol.append(data_dict)

# collate energies into DataFrame        
df = pd.DataFrame.from_records(data_by_mol)
df = df.sort_values('Molecule')

# round energy values down to desired precision
round_fn = lambda x : round(x, PRECISION)

for col_name, col in df.items():
    try:
        df[col_name] = col.apply(round_fn) # attempt to round column and replace with rounded values
    except TypeError:
        pass

# save energies to file
energy_file = omm_path / 'energies_1_step.csv'
df.to_csv(energy_file, index=False)

In [137]:
df.reset_

,Chemistry,Molecule,Potential Energy (kJ/mol),Potential Energy (kcal/mol),Kinetic Energy (kJ/mol),Kinetic Energy (kcal/mol)
0,NIPU,NIPU_11,1256.003,300.192,6.943,1.659
1,NIPU,NIPU_2,302.227,72.234,13.951,3.334
16,urethane,urethane_0,4355.840,1041.071,13.791,3.296
9,urethane,urethane_1,-1396.220,-333.704,6.962,1.664
15,urethane,urethane_10,2262.249,540.690,6.767,1.617
14,urethane,urethane_14,3613.522,863.652,10.349,2.474
11,urethane,urethane_15,1234.156,294.970,6.683,1.597
21,urethane,urethane_17,-4639.427,-1108.850,12.257,2.929
17,urethane,urethane_2,3438.307,821.775,10.594,2.532
12,urethane,urethane_22,10704.400,2558.413,14.111,3.373


In [ ]:
s = sim.context.getState(getEnergy=True)
ser = XmlSerializer.serialize(s)
print(ser)

In [ ]:
df = pd.read_csv(sim_paths.state_data)
df

In [ ]:
sim_params

In [ ]:
state_params : dict[str, bool] = {
    'getPositions'  : True,
    'getVelocities' : True,
    'getForces'     : False,
    'getEnergy'     : True,
    'getParameters' : True,
    'getParameterDerivatives' : False,
    'getIntegratorParameters' : False
}

state = sim.context.getState(**state_params)

In [ ]:
from openmm import XmlSerializer

with (mol_dir / 'system.xml').open('w') as file:
    ser_str = XmlSerializer.serialize(sim.system)
    file.write(ser_str)

with (mol_dir / 'state.xml').open('w') as file:
    ser_str = XmlSerializer.serialize(state)
    file.write(ser_str)

In [ ]:
for force in sim.system.getForces():
    print(force.getName(), force.getForceGroup())

In [ ]:
help(sim.context.getState)

In [ ]:
state.getEne

In [ ]:
dir(force)

# Inspecting which molecules fail and succeed and why

In [ ]:
import re
regex = re.compile(r'\A(\w+_\d+)')

diff = {}
for chemistry, succ_names in success_ics.items():
    chem_dir = coll_path / chemistry
    for sdf_path in chem_dir.glob('**/*.sdf'):
        mol_name = re.search(regex, sdf_path.stem).group()
        if mol_name not in succ_names:
            diff.append(mol_name)

## Checking for successful residue covers of newly-generated PDB Topologies

In [ ]:
chemistry = 'urethane'

mgr = PolymerManager(coll_path / chemistry)
mol_names = failed_interchanges[chemistry]['UnmatchedAtomsError']

offmols = {
    mol_name : mgr.polymers[mol_name].offmol_matched(strict=False)
        for mol_name in mol_names
}

In [ ]:
sizes = {
    mol_name : offmol.n_atoms
        for mol_name, offmol in sorted(offmols.items(), key=lambda x : x[1].n_atoms)
}

In [ ]:
sizes

In [ ]:

pdir = mgr.polymers['urethane_6']
# pdir = mgr.polymers['NIPU_8']
# pdir.offmol_matched(strict=True)

for atom in offmol.atoms:
    if not atom.metadata['already_matched']:
        print(atom.metadata)

In [ ]:
mgr = PolymerManager(coll_path / 'NIPU')

offmols = {}
unmatched = []
for mol_name, polymer in mgr.polymers.items():
    try:
        offmols[mol_name] = polymer.offmol
    except:
        unmatched.append(mol_name)

In [ ]:
for mol_name in unmatched:
    print(mol_name)
    polymer = mgr.polymers[mol_name]
    offmol = polymer.offmol_matched(strict=False)
    
    for atom in offmol.atoms:
        if not atom.metadata['already_matched']:
            print('\t', atom.metadata)

# Experimenting with SDF files

In [ ]:
benz = Chem.MolFromSmiles('C1ccccC=1')
benz = Chem.AddHs(benz)
benz.SetDoubleProp('stuff', 3.14)
benz

In [ ]:
block2k = Chem.MolToMolFile(benz, 'test_2k.sdf')
block3k = Chem.MolToV3KMolFile(benz, 'test_3k.sdf')

In [ ]:
block2kforce = Chem.MolToMolFile(benz, 'test_2k_force.sdf', forceV3000=True)

In [ ]:
with Chem.SDWriter('test_sdw.sdf') as sdwriter:
    sdwriter.SetForceV3000(True)
    print(sdwriter.GetForceV3000())

    sdwriter.write(benz)

In [ ]:
with Chem.SDMolSupplier('sdf_testing/test_off_rd.sdf', sanitize=False) as suppl:
    mols = [mol for mol in suppl]

targ = mols[0]
targ

In [ ]:
omol = Molecule.from_rdkit(benz)
omol.generate_conformers(n_conformers=1)
omol.visualize(backend='nglview')

In [ ]:
from polysaccharide import filetree
from polysaccharide import TOOLKITS


p = Path('sdf_testing/test_off_rd.sdf')
tkwrap = TOOLKITS['The RDKit']

omol.properties['series'] = (1,2,3)
omol.to_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load = Molecule.from_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load.properties

In [ ]:
help(tkwrap.to_file)

In [ ]:
help(omol.to_file)